In [ ]:
#EXTRACCION TRANSFORMACION Y CARGA DE DATOS (ETL)

#Importar las librerías para el análisis de los datos
import requests
import pandas as pd


#Leer los datos directamente desde GitHub

#Archivos json
url="https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/netflix_titles.json"
f=requests.get(url)
data=f.json()

# Convertir el archivo de json en un DataFrame
netflix=pd.DataFrame(data)

#Archivos csv
amazon= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/amazon_prime_titles.csv")
disney= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/disney_plus_titles.csv")
hulu= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/hulu_titles.csv") 

#Agregamos la columna de plataform y unimos las tablas y las unimos
netflix["plataform"]= "netflix"
amazon["plataform"]="amazon"
disney["plataform"]="disney"
hulu["plataform"]="hulu"


#Unimos los dataframes
df=pd.concat([netflix, amazon, disney, hulu]) 


#Arreglo de la base de datos de la columna duración
#Creamos un dataframe donde guardaremos los datos de duración que estaban ubicados en la columna rating
data_all=pd.DataFrame([])

#Identificamos la lista de indicadores de data que es de duración
Lista_wrong=["min", "Seasons","Season"]

#Filtramos el dataframe principal con las columnas que queremos arreglar
df_analysis= df[["show_id","rating","plataform"]]


#Realizamos un ciclo iterativo para identificar los datos de la columna rating que debería ir en duración
for elemento in Lista_wrong:
    
    # Filtrar el Dataframe por columna rating en el que se encuentre la palabra min, Seasons, Season
    df_analysis_filter= df_analysis.loc[df_analysis["rating"].str.contains(elemento, na=False)]
    data_all=pd.concat([data_all,df_analysis_filter])
    
#Renombramos la columna    
data_all.rename(columns={'rating':"duration"}, inplace=True)

#Identificamos de la columna de duración del dataframe principal cuáles eran sus valores no nulos
#Sacamos las variables del dataframe principal
df_analysis_2= df[["show_id","duration","plataform"]]
df_analysis_2=df_analysis_2.dropna(subset="duration")

#Juntamos todos los datos de duración en una sóla columna
data_all=pd.concat([data_all, df_analysis_2])
data_all

#Guardamos todos esos valores en otra columna para no sobreponer y/o perder información
data_all.rename(columns={'duration':"duration_2"}, inplace=True)

#Quiamos las cadenas de texto para tener valores numéricos en duración y poder analizar correctamente los valores maximos
data_all["duration_2"]=data_all["duration_2"].str.replace(" min","")
data_all["duration_2"]=data_all["duration_2"].str.replace(" Seasons","")
data_all["duration_2"]=data_all["duration_2"].str.replace(" Season","")

#Juntamos todo en la tabla principal
df_final= pd.merge(df,data_all, on=["show_id","plataform"],how="left")

df_final


In [158]:
df_types=pd.DataFrame([])
df_types["type"]=pd.DataFrame(df_final["type"].unique())
df_types["type_id"]=1,2
df_types= df_types.set_index("type")
df_types.to_csv("types")
df_types

,type_id
type,
Movie,1
TV Show,2


In [197]:
df_shows= df_final.drop_duplicates(subset=["title","type"])
df_shows=df_shows[["title","type","director","cast","country","release_year"]]
df_shows["title_id"]= df_shows.reset_index().index
df_shows["cast"]=df_shows["cast"].str.replace(",",";")
df_shows= pd.merge(df_shows,df_types, on="type",how="left")
df_shows=df_shows.set_index(["title","type"])
df_shows.to_csv("shows")
df_shows


,,director,cast,country,release_year,title_id,type_id
title,type,,,,,,
Dick Johnson Is Dead,Movie,Kirsten Johnson,None,NaN,2020,0,1
Blood & Water,TV Show,None,Ama Qamata; Khosi Ngema; Gail Mabalane; Thaban...,NaN,2021,1,2
Ganglands,TV Show,Julien Leclercq,Sami Bouajila; Tracy Gotoas; Samuel Jouy; Nabi...,NaN,2021,2,2
Jailbirds New Orleans,TV Show,None,None,NaN,2021,3,2
Kota Factory,TV Show,None,Mayur More; Jitendra Kumar; Ranjan Raj; Alam K...,NaN,2021,4,2
...,...,...,...,...,...,...,...
Samurai Harem,TV Show,NaN,NaN,NaN,2009,22237,2
Scream Queens,TV Show,NaN,NaN,NaN,2015,22238,2
Star Trek: The Original Series,TV Show,NaN,NaN,NaN,1966,22239,2


In [189]:
df_platform=pd.DataFrame([])
df_platform["plataform"]=pd.DataFrame(df_final["plataform"].unique())
df_platform["platform_id"]= df_platform.reset_index().index
df_platform=df_platform.set_index("plataform")
df_platform.to_csv("plataform")

df_platform

,platform_id
plataform,
netflix,0
amazon,1
disney,2
hulu,3


In [179]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23018 entries, 0 to 23017
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       23018 non-null  object 
 1   type          23018 non-null  object 
 2   title         23018 non-null  object 
 3   director      14739 non-null  object 
 4   cast          17677 non-null  object 
 5   country       0 non-null      float64
 6   date_added    13464 non-null  object 
 7   release_year  23018 non-null  int64  
 8   rating        22154 non-null  object 
 9   duration      22516 non-null  object 
 10  listed_in     23018 non-null  object 
 11  description   23014 non-null  object 
 12  plataform     23018 non-null  object 
 13  duration_2    22831 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.6+ MB


In [215]:
df_shows_in_platform=df_final[["title","type","plataform","show_id","duration_2","rating","date_added","listed_in"]]
df_shows_in_platform=pd.merge(df_shows_in_platform,df_platform, on="plataform",how="left")

df_shows_in_platform=pd.merge(df_shows_in_platform,df_types, on="type",how="left")


df_shows_in_platform=df_shows_in_platform.set_index(["title","type"])
df_shows_in_platform

df_shows_in_platform=pd.merge(df_shows_in_platform,df_shows[["title_id"]], on=["title","type"],how="left")

df_shows_in_platform


,,plataform,show_id,duration_2,rating,date_added,listed_in,platform_id,type_id,title_id
title,type,,,,,,,,,
Dick Johnson Is Dead,Movie,netflix,s1,90,PG-13,"September 25, 2021",Documentaries,0,1,0
Blood & Water,TV Show,netflix,s2,2,TV-MA,"September 24, 2021","International TV Shows, TV Dramas, TV Mysteries",0,2,1
Ganglands,TV Show,netflix,s3,1,TV-MA,"September 24, 2021","Crime TV Shows, International TV Shows, TV Act...",0,2,2
Jailbirds New Orleans,TV Show,netflix,s4,1,TV-MA,"September 24, 2021","Docuseries, Reality TV",0,2,3
Kota Factory,TV Show,netflix,s5,2,TV-MA,"September 24, 2021","International TV Shows, Romantic TV Shows, TV ...",0,2,4
...,...,...,...,...,...,...,...,...,...,...
Star Trek: The Original Series,TV Show,hulu,s3069,3,TV-PG,NaN,"Action, Adventure, Classics",3,2,22239
Star Trek: Voyager,TV Show,hulu,s3070,7,TV-PG,NaN,"Action, Adventure, Science Fiction",3,2,5245
The Fades,TV Show,hulu,s3071,1,TV-14,NaN,"Horror, International, Science Fiction",3,2,9464


In [217]:
df_shows_in_platform=df_shows_in_platform.reset_index()
df_shows_in_platform=df_shows_in_platform.set_index(["title_id","type_id","platform_id"])
df_shows_in_platform

,,,index,title,type,plataform,show_id,duration_2,rating,date_added,listed_in
title_id,type_id,platform_id,,,,,,,,,
0,1,0,0,Dick Johnson Is Dead,Movie,netflix,s1,90,PG-13,"September 25, 2021",Documentaries
1,2,0,1,Blood & Water,TV Show,netflix,s2,2,TV-MA,"September 24, 2021","International TV Shows, TV Dramas, TV Mysteries"
2,2,0,2,Ganglands,TV Show,netflix,s3,1,TV-MA,"September 24, 2021","Crime TV Shows, International TV Shows, TV Act..."
3,2,0,3,Jailbirds New Orleans,TV Show,netflix,s4,1,TV-MA,"September 24, 2021","Docuseries, Reality TV"
4,2,0,4,Kota Factory,TV Show,netflix,s5,2,TV-MA,"September 24, 2021","International TV Shows, Romantic TV Shows, TV ..."
...,...,...,...,...,...,...,...,...,...,...,...
22239,2,3,23013,Star Trek: The Original Series,TV Show,hulu,s3069,3,TV-PG,NaN,"Action, Adventure, Classics"
5245,2,3,23014,Star Trek: Voyager,TV Show,hulu,s3070,7,TV-PG,NaN,"Action, Adventure, Science Fiction"
9464,2,3,23015,The Fades,TV Show,hulu,s3071,1,TV-14,NaN,"Horror, International, Science Fiction"


In [ ]:
df_shows_in_platform=df_shows_in_platform.drop(["index","title","type","plataform"], axis=1)


In [223]:
df_shows_in_platform.rename(columns={"duration_2":"duration"})
df_platform.to_csv("shows_in_platform")


In [210]:
df_shows[["title_id"]]

,,title_id
title,type,
Dick Johnson Is Dead,Movie,0
Blood & Water,TV Show,1
Ganglands,TV Show,2
Jailbirds New Orleans,TV Show,3
Kota Factory,TV Show,4
...,...,...
Samurai Harem,TV Show,22237
Scream Queens,TV Show,22238
Star Trek: The Original Series,TV Show,22239


In [141]:
df_duplicados=df_final[df_final[["title","type"]].duplicated()]
list_duplicados=df_duplicados["title"].unique()

df_shows= df_final.drop_duplicates(subset=["title","type"])

df_shows=pd.DataFrame(df_final["title"].unique())
df_shows["index"]= df_shows.reset_index().index
df_shows=df_shows.rename(columns={0:"title", "index":"title_id"})
df_shows=df_shows.set_index("title_id")
df_shows

,title
title_id,
0,Dick Johnson Is Dead
1,Blood & Water
2,Ganglands
3,Jailbirds New Orleans
4,Kota Factory
...,...
22110,Samurai Harem
22111,Scream Queens
22112,Star Trek: The Original Series


In [35]:
df_shows=df_final[["title", "type", "director","cast","country","release_year"]]
null_counts=df_shows.isnull().sum()
null_counts[null_counts>0].sort_values(ascending=False)

country     11499
director     8279
cast         5341
dtype: int64

In [36]:
lista_peliculas=df_final["title"].unique()

for elemento in lista_peliculas:
    df_shows[df_shows["title"]==elemento]

array(['Dick Johnson Is Dead', 'Blood & Water', 'Ganglands', ...,
       'Star Trek: The Original Series', 'The Twilight Zone',
       'Tokyo Magnitude 8.0'], dtype=object)

In [39]:
df_shows[df_shows["title"]=="Blood & Water"]

,title,type,director,cast,country,release_year
1,Blood & Water,TV Show,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021


In [ ]:
#CONSULTAS

#Maxima duracion segun tipo de fil, por plataforma, y por año
def get_max_duration(year, plataform, tipo):
    a=2
    
#Cantidad de peliculas y series (separado) POR PLATAFORMA
def get_count_plataform(plataforma):
    a=3
    
#Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo
def get_listein(genero):
    a=4
    
#Actor que más se repite según plataforma y año
def get_actor(plataforma, year):
    a=5

In [66]:
duplicate_movies=df_final[df_final.duplicated()]
duplicate_movies

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
19976,s51,Movie,Yashahime: Princess Half-Demon,NaN,NaN,Japan,"October 9, 2021",2020,2 Seasons,NaN,Anime,Long-lost twins Towa and Setsuna reunite after...,hulu,2
20190,s264,Movie,Trolls: TrollsTopia,NaN,NaN,United States,"September 2, 2021",2020,4 Seasons,NaN,"Family, Kids",Trolls: TrollsTopia! is the next chapter in th...,hulu,4
20399,s472,Movie,This Way Up,NaN,NaN,United Kingdom,"July 9, 2021",2019,2 Seasons,NaN,"Comedy, Drama, International","This Way Up is a comedy drama about moving on,...",hulu,2
20730,s802,Movie,Everything's Gonna Be Okay,NaN,NaN,United States,"April 9, 2021",2020,2 Seasons,NaN,"Comedy, Drama",Australian twenty-something Nicholas is left t...,hulu,2
20777,s848,Movie,Juda,NaN,NaN,Israel,"March 30, 2021",2017,2 Seasons,NaN,"Action, Adventure, Crime",Juda is a low-life gambler hustling a living i...,hulu,2
20859,s929,Movie,Blippi,NaN,NaN,United States,"March 1, 2021",2016,3 Seasons,NaN,Kids,Explore the world as seen through the eyes of ...,hulu,3
20919,s988,Movie,Harrow,NaN,NaN,"United States, Australia","February 8, 2021",2018,3 Seasons,NaN,"Crime, Drama, International",Daniel Harrow is a brilliant forensic patholog...,hulu,3
20985,s1053,Movie,Celebs Go Dating,NaN,NaN,United Kingdom,"January 8, 2021",2016,3 Seasons,NaN,Reality,The Celebrity Dating Agency opens its doors to...,hulu,3
21069,s1136,Movie,Brassic,NaN,NaN,United Kingdom,"December 4, 2020",2019,2 Seasons,NaN,Comedy,"Vinnie, a Lancashire lad with bipolar disorder...",hulu,2
21202,s1268,Movie,Chicken Girls,NaN,NaN,United States,"October 31, 2020",2017,6 Seasons,NaN,"Drama, Kids, Teen",Rhyme and her friends have been dancing togeth...,hulu,6


In [72]:
df_sorted=df_final[df_final["country"].isnull()].sort_values("title")

In [113]:
#Obtener lista de duplicados
df_duplicados=df_final[df_final[["title","type"]].duplicated()]
list_duplicados=df_duplicados["title"].unique()
list_duplicados

data_total=pd.DataFrame([])
for elemento in list_duplicados:
    
    #Filtrar dataframe y ordener por categoría de country para cada pelicula
    df_prueba_fill=df_final[df_final["title"]==elemento].sort_values("country")
    df_final["country"]=df_prueba_fill["country"].fillna(method="ffill")
    data_total= pd.concat([data_total,df_prueba_fill])

data_total
    

    

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
2575,s2576,TV Show,WWII in HD,None,"Gary Sinise, Rob Lowe, Josh Lucas, Justin Bart...",United States,"May 2, 2020",2009,TV-14,1 Season,Docuseries,Rare color footage interwoven with firsthand i...,netflix,1
8868,s62,TV Show,WWII in HD,NaN,"Gary Sinise, Charles Scheffel",NaN,NaN,2009,TV-14,1 Season,Action,The only people to see the war like this were ...,amazon,1
4228,s4229,Movie,Whitney Cummings: Money Shot,John Fortenberry,Whitney Cummings,NaN,"January 1, 2019",2010,TV-MA,48 min,Stand-Up Comedy,Comedy Central roast veteran Whitney Cummings ...,netflix,48
8918,s112,Movie,Whitney Cummings: Money Shot,John Fortenberry,Whitney Cummings,NaN,NaN,2010,NaN,48 min,"Arts, Entertainment, and Culture, Comedy, Spec...",Money Shot finds Cummings riffing on a wide ra...,amazon,48
2385,s2386,Movie,Underdogs,Juan José Campanella,"Ariana Grande, Nicholas Hoult, Matthew Morriso...",NaN,"June 15, 2020",2013,PG,85 min,"Children & Family Movies, Sports Movies","When his archenemy returns to settle a score, ...",netflix,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22991,s3047,TV Show,Arrested Development,NaN,NaN,NaN,NaN,2003,TV-14,3 Seasons,"Comedy, Sitcom","In this critically acclaimed series, Michael B...",hulu,3
6795,s6796,TV Show,Frasier,None,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",NaN,NaN,2003,TV-PG,11 Seasons,"Classic & Cult TV, TV Comedies",Frasier Crane is a snooty but lovable Seattle ...,netflix,11
22998,s3054,TV Show,Frasier,NaN,NaN,NaN,NaN,1993,TV-PG,11 Seasons,"Comedy, Sitcom","Comedy series set in Seattle, WA, which chroni...",hulu,11
9478,s672,TV Show,The Fades,NaN,"Iain De Caestecker, Daniel Kaluuya, Tom Ellis,...",NaN,NaN,2011,TV-14,1 Season,Drama,"A smart, supernatural thriller about two frien...",amazon,1


In [114]:
df_final[df_final["title"]=="21"]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
5959,s5960,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",NaN,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,netflix,123
15861,s7055,Movie,21,Robert Luketic,"Jim Sturgess, Kate Bosworth, Aaron Yoo, Liza L...",NaN,NaN,2008,PG-13,123 min,"Drama, Suspense",Academy Award Winner Kevin Spacey (The Usual S...,amazon,123
20311,s385,Movie,21,NaN,NaN,NaN,"August 1, 2021",2008,PG-13,123 min,Drama,"Ben Campbell (Jim Sturgess), a brilliant stude...",hulu,123


In [109]:
data_total[data_total["title"]=="21"]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
5959,s5960,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",NaN,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,netflix,123
15861,s7055,Movie,21,Robert Luketic,"Jim Sturgess, Kate Bosworth, Aaron Yoo, Liza L...",NaN,NaN,2008,PG-13,123 min,"Drama, Suspense",Academy Award Winner Kevin Spacey (The Usual S...,amazon,123
20311,s385,Movie,21,NaN,NaN,NaN,"August 1, 2021",2008,PG-13,123 min,Drama,"Ben Campbell (Jim Sturgess), a brilliant stude...",hulu,123


In [73]:
df_sorted[df_sorted["title"].duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
16280,s7474,Movie,1 Night in San Diego,Penelope Lawson,"Laura Ashley Samuels, Jenna Ushkowitz, Alexand...",NaN,NaN,2020,18+,86 min,Comedy,"BFFs Hannah and Brooklyn, played by Jenna Ushk...",amazon,86
20311,s385,Movie,21,NaN,NaN,NaN,"August 1, 2021",2008,PG-13,123 min,Drama,"Ben Campbell (Jim Sturgess), a brilliant stude...",hulu,123
17024,s8218,Movie,Attack of the 50 Foot Cheerleader,Kevin O'Neill,"Sean Young, Ryan Merriman, Treat Williams, Joh...",NaN,NaN,2012,R,83 min,Comedy,Cheerleader Cassie Stratford downs an experime...,amazon,83
21250,s1316,Movie,Bad Hombres,NaN,NaN,NaN,"October 16, 2020",2020,TV-14,NaN,"Documentaries, Sports","Chronicling the Tecolotes de los Dos Laredos, ...",hulu,NaN
2101,s2102,TV Show,Baewatch: Parental Guidance,None,None,None,"August 22, 2020",2019,TV-MA,1 Season,"British TV Shows, Reality TV, Romantic TV Shows",Couples go on a romantic getaway to test their...,netflix,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18632,s158,Movie,What the Shark?,NaN,Robert Davi,NaN,"July 23, 2021",2020,TV-PG,44 min,"Animals & Nature, Documentary",What the Shark? dives deep into the environmen...,disney,44
21546,s1610,TV Show,Why We Fight,NaN,NaN,NaN,"June 9, 2020",2017,NaN,2 Seasons,"Documentaries, Sports",Why We Fight follows Cat Zingano as she explor...,hulu,2
13231,s4425,TV Show,Word of Honor,NaN,"Zhehan Zhang, Jun Gong, Ye Zhou, Wenyuan Ma, X...",NaN,"April 17, 2021",2021,18+,2 Seasons,"Action, Drama, Romance",(Starting from EP 19 and after) Zishu Zhou (Zh...,amazon,2
14873,s6067,Movie,Worth,Jenn Page,Eric Roberts,NaN,NaN,2021,13+,90 min,"Drama, Special Interest, Suspense",Johnny was a young seminary student who lost h...,amazon,90


In [85]:
df_prueba_fill=df_final[df_final["title"]=="21"].sort_values("country")
df_prueba_fill


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
5959,s5960,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,netflix,123
15861,s7055,Movie,21,Robert Luketic,"Jim Sturgess, Kate Bosworth, Aaron Yoo, Liza L...",NaN,NaN,2008,PG-13,123 min,"Drama, Suspense",Academy Award Winner Kevin Spacey (The Usual S...,amazon,123
20311,s385,Movie,21,NaN,NaN,NaN,"August 1, 2021",2008,PG-13,123 min,Drama,"Ben Campbell (Jim Sturgess), a brilliant stude...",hulu,123


In [90]:
df_prueba_fill["country"]=df_prueba_fill["country"].fillna(method="ffill")
df_prueba_fill

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
5959,s5960,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,netflix,123
15861,s7055,Movie,21,Robert Luketic,"Jim Sturgess, Kate Bosworth, Aaron Yoo, Liza L...",United States,NaN,2008,PG-13,123 min,"Drama, Suspense",Academy Award Winner Kevin Spacey (The Usual S...,amazon,123
20311,s385,Movie,21,NaN,NaN,United States,"August 1, 2021",2008,PG-13,123 min,Drama,"Ben Campbell (Jim Sturgess), a brilliant stude...",hulu,123


In [65]:
df_final["country"].isnull()


0        False
1        False
2         True
3         True
4        False
         ...  
23013    False
23014    False
23015    False
23016    False
23017    False
Name: country, Length: 23018, dtype: bool